In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Veri yüklenir ve işlenir
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")
df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Veriyi eğitim ve test setlerine ayırma
train_size = int(len(df) * 0.80)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Özelliklerin ve hedef değişkenin ayrılması
def create_features(df):
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    return df

train = create_features(train)
test = create_features(test)

FEATURES = ['dayofweek', 'quarter', 'month', 'year', 'dayofyear']
TARGET = 'İstanbul günlük tüketim(m³/gün)'

X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

# Veri normalleştirme
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Veriyi 3 boyutlu hale getirme
X_train_scaled_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_scaled_reshaped = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# BiLSTM modeli oluşturulur
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_scaled_reshaped.shape[1], 1)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1))

# Modelin derlenmesi
model.compile(optimizer='adam', loss='mean_squared_error')

# EarlyStopping geri çağırmasını tanımlama
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=60, 
                       verbose=1, mode='auto', restore_best_weights=True)

# Modelin eğitimi
history = model.fit(X_train_scaled_reshaped, y_train, 
                    validation_data=(X_test_scaled_reshaped, y_test),
                    callbacks=[monitor],
                    verbose=1, 
                    epochs=100,
                    batch_size=32)

# Test veri seti üzerinde tahmin yapılır
y_pred = model.predict(X_test_scaled_reshaped)

# Tahminlerin gerçek değerlerle karşılaştırılması
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)



print("Mean Squared Error (MSE): {:.4f}".format(mse))
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))
print("Mean Absolute Error (MAE): {:.4f}".format(mae))
print("Mean Absolute Percentage Error (MAPE): {:.4f}".format(mape*100))

C:\Users\DOGUKAN\AppData\Local\Temp\ipykernel_6600\4106279778.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = df.index.dayofweek
C:\Users\DOGUKAN\AppData\Local\Temp\ipykernel_6600\4106279778.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quarter'] = df.index.quarter
C:\Users\DOGUKAN\AppData\Local\Temp\ipykernel_6600\4106279778.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Epoch 1/100
118/118 [==============================] - 9s 23ms/step - loss: 20.9459 - val_loss: 0.0272
Epoch 2/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0435 - val_loss: 0.0148
Epoch 3/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0232 - val_loss: 0.0111
Epoch 4/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0128 - val_loss: 0.0108
Epoch 5/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0096 - val_loss: 0.0128
Epoch 6/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0085 - val_loss: 0.0119
Epoch 7/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0079 - val_loss: 0.0119
Epoch 8/100
118/118 [==============================] - 1s 10ms/step - loss: 0.0076 - val_loss: 0.0163
Epoch 9/100
118/118 [==============================] - 1s 11ms/step - loss: 0.0073 - val_loss: 0.0134
Epoch 10/100
118/118 [==============================] - 1s 11ms/step - loss: 0.00